# 2. QAT

QAT概述：与其他量化方法相比，QAT在训练过程中模拟量化的效果，可以获得更高的accuracy. 在训练过程中，所有的计算都是再浮点上进行的，使用fake_quant模块通过夹紧和摄入的方式对量化效果进行建模，模拟int8的效果。模型转换后，权值和激活被量化，激活在可能的情况下被融合到前一层。它通常与CNN一起使用，与PTQ相比具有更高的accuracy.

1)  量化模型

In [5]:
import torch
from torch import nn, Tensor


class M(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = torch.nn.Conv2d(1, 1, 1)
        self.relu = torch.nn.ReLU()
        
    def _forward_impl(self, x:Tensor) -> Tensor:
        """提供便捷函数"""
        x = self.conv(x)
        x = self.relu(x)
        return x
    
    def forward(self, x:Tensor) -> Tensor:
        x = self._forward_impl(x)
        return x

from torch.ao.quantization import QuantStub, DeQuantStub

class QM(M):
    """_summary_

    Args:
        M (_type_): _description_
        is_print: 为了测试需求，打印一些信息
    """
    def __init__(self, is_print: bool=False):
        super().__init__()
        self.is_print = is_print
        self.quant = QuantStub()  # 将张量从浮点转换为量化
        self.dequant = DeQuantStub()  # 将张量从量化转换为浮点
        
    def forward(self, x:Tensor) -> Tensor:
        # 手动指定张量将在量化模型中从浮点模块转换为量化模块的位置；
        x = self.quant(x)
        if self.is_print:
            print("量化前的类型：", x.dtype)
        x = self._forward_impl(x)
        if self.is_print:
            print("量化中的类型：", x.dtype)
        # 在量化模型中手动指定张量从量化到浮点的转换位置；
        x = self.dequant(x)
        if self.is_print:
            print("量化后的类型：", x.dtype)
        return x

In [3]:
class M2(M):
    def __init__(self):
        super().__init__()
        self.bn = torch.nn.BatchNorm2d(1)
    
    def __forward_impl(self, x: Tensor) -> Tensor:
        """提供便捷函数"""
        x = self.conv(x)
        x = self.bn(x)
        x = self.relu(x)
        return x
    

In [6]:
# 定义可量化模块
class QM2(M2, QM):
    def __init__(self):
        super().__init__()

In [8]:
# 创建浮点模型实例
model_fp32 = QM2()
model_fp32

QM2(
  (conv): Conv2d(1, 1, kernel_size=(1, 1), stride=(1, 1))
  (relu): ReLU()
  (quant): QuantStub()
  (dequant): DeQuantStub()
  (bn): BatchNorm2d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

2)  模型必须设置为训练模式，以便QAT使用

In [9]:
model_fp32.train()

QM2(
  (conv): Conv2d(1, 1, kernel_size=(1, 1), stride=(1, 1))
  (relu): ReLU()
  (quant): QuantStub()
  (dequant): DeQuantStub()
  (bn): BatchNorm2d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [10]:
# 添加量化配置(与PTQ相同相似)
model_fp32.qconfig = torch.ao.quantization.get_default_qat_qconfig('fbgemm')

In [11]:
# 融合QAT模块
# QAT的模块融合与PTQ相同相似
from torch.ao.quantization import fuse_modules_qat
model_fp32_fused = fuse_modules_qat(model_fp32,[['conv', 'bn', 'relu']])

In [14]:
# 准备QAT模型
model_fp32_prepared = torch.quantization.prepare_qat(model_fp32_fused)
# 训练QAT模型
def training_loop(para):
    pass
training_loop(model_fp32_prepared)


/home/zhangyouan/anaconda3/envs/stc/lib/python3.10/site-packages/torch/ao/quantization/observer.py:214: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(


In [15]:
# 将观测到的模型转换为量化模型。需要：
# 1. 量化权重，计算和存储用于每个激活张量的尺度（scale)和偏差（bias)值，
# 2. 在适当的地方融合模块，并用量化实现替换关键算子；
model_fp32_prepared.eval()
model_int8 = torch.quantization.convert(model_fp32_prepared)

/home/zhangyouan/anaconda3/envs/stc/lib/python3.10/site-packages/torch/ao/quantization/utils.py:310: UserWarning: must run observer before calling calculate_qparams. Returning default values.
  warnings.warn(


In [17]:
# 运行模型，相关的计算将在{data}torch.qint8中发生
input_fp32 = torch.randn(4, 1, 4, 4)
res = model_int8(input_fp32)